In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
from IPython import display

In [2]:
LR = 0.001
BATCH_SIZE = 50
TIME_STEP = 28 # rnn time step / image height
INPUT_SIZE = 28 # rnn input size / image width
EPOCH = 1
DOWNLOAD_MNIST = False

In [3]:
train_data = torchvision.datasets.MNIST(
    root='./mnist',
    train=True,
    transform=torchvision.transforms.ToTensor(), 
    download=DOWNLOAD_MNIST
)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)
test_x = Variable(torch.unsqueeze(test_data.data, dim=1)).type(torch.float32)[:2000].squeeze() / 255
test_y = Variable(test_data.targets[:2000])

In [4]:
test_x.size()

torch.Size([2000, 28, 28])

In [5]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(
            input_size=INPUT_SIZE,
            hidden_size=64, # 隐藏层节点数
            num_layers=1, # LSTM隐藏层层数
            batch_first=True, # (batch, time_step, input)
        )

        self.out = nn.Linear(64, 10)

    def forward(self, x):
        # h_n, h_c是最后一个时刻的hidden state
        r_out, (h_n, h_c) = self.rnn(x, None) # 第一个hidden state没有
        # 因为结构是(batch, time_step, input)，所以-1选取最后一个时刻
        out = self.out(r_out[:, -1, :]) # 选取最后一个时刻的output
        return out

rnn = RNN()
print(rnn)

RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)


In [7]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

for epoch in range(EPOCH):
    for step, (batch_x, batch_y) in enumerate(train_loader):
        b_x = Variable(batch_x.view(-1, 28, 28))
        print(b_x.size())
        b_y = Variable(batch_y)
        output = rnn(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward() # backpropagation
        optimizer.step()

        if step % 50 == 0:
            test_output = rnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (sum(pred_y == test_y) / test_y.size()[0]).item()
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.item(), '| test accuracy: ', accuracy)

# print 10 prediction2 from test data
torch_output = rnn(test_x[:10])
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
print(pred_y[:10], 'prediction number')
print(test_y[:10].numpy(), 'real number')

torch.Size([50, 28, 28])
Epoch:  0 | train loss: 0.4820 | test accuracy:  0.828499972820282
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])
torch.Size([50, 28, 28])


KeyboardInterrupt: 